# log TR vs log EIRPmin Taking into account multiple bands

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

def plot_logTR_vs_logEIRPmin(
    catalog_nested_dict,
    compare_mode='per_band_across_approaches',
    bands_to_compare=None,
    approaches_to_compare=None,
    overlay=True,
    colors=None,
    markers=None,
    show=True,
    return_slopes=False
):
    """
    Flexible comparative plot of log(TR) vs log(EIRPmin) for multiple catalogs and bands.

    Parameters
    ----------
    catalog_nested_dict : dict
        Dict of form: {
            'Gaia': {'L': shell_results_L, 'S': shell_results_S, ...} or {'all': shell_results_all},
            'SynthPop': {...}, 'NED': {...}, 'Uno': {...}
        }
        Each sub-dictionary may contain one or more bands as keys.
    compare_mode : str
        - 'per_band_across_approaches': Compare one band in several approaches (e.g. GBT-L in Gaia/SynthPop/NED/Uno).
        - 'per_approach_across_bands': Compare several bands within one approach (e.g. L/S/X in Gaia).
        - 'combined_across_approaches': Compare everything combined from all fields per approach (split_by_band=False).
    bands_to_compare : list of str or None
        List of bands (e.g. ['L', 'S']) to include if using per_band comparison.
    approaches_to_compare : list of str or None
        List of approaches (e.g. ['Gaia', 'SynthPop']) to include. If None, uses all present.
    overlay : bool
        If True (default), plot all selected on one figure with legend; else, returns separate (fig,ax) for each curve.
    colors, markers : optional dicts for color/marker styling (see docstring)
    show : bool
        Display figure(s).
    return_slopes : bool
        If True, also return dict of slopes for each plotted set.

    Returns
    -------
    (fig, ax) if overlay is True, else list of (fig, ax)
    Optionally slopes_dict if return_slopes is True.
    """
    if approaches_to_compare is None:
        approaches_to_compare = list(catalog_nested_dict.keys())

    if colors is None:
        seq = plt.colormaps["tab10"].colors
        colors = {}
        for i, app in enumerate(approaches_to_compare):
            for j, band in enumerate(bands_to_compare or ['all']):
                key = (app, band)
                colors[key] = seq[(i*2+j) % len(seq)]
    if markers is None:
        base_markers = ['o', '^', 's', 'v', 'd', 'x']
        markers = {}
        for i, app in enumerate(approaches_to_compare):
            for j, band in enumerate(bands_to_compare or ['all']):
                markers[(app, band)] = base_markers[(i+j) % len(base_markers)]

    if overlay:
        fig, ax = plt.subplots(figsize=(8, 6))
    else:
        figs, axs = [], []

    slopes_dict = {}

    vlines = [(13, 'darkblue', 'Arecibo Radar'), (16, 'magenta', 'Kardashev I'),
                (26, 'orange', 'Kardashev II'), (36, 'darkred', 'Kardashev III')]

    def get_eirp_col(approach):
        return 'log_EIRPmin_shell' if approach.lower() in ['gaia','synthpop'] else 'log_EIRPmin'

    if compare_mode == 'per_band_across_approaches':
        if not bands_to_compare:
            bands_to_compare = list(next(iter(catalog_nested_dict.values())).keys())
        for band in bands_to_compare:
            all_x, all_y = [], []
            for app in approaches_to_compare:
                if band not in catalog_nested_dict[app]: continue
                df = catalog_nested_dict[app][band]
                eirp_col = get_eirp_col(app)
                x = df[eirp_col].to_numpy()
                y = df['log_TR'].to_numpy()
                mask = np.isfinite(x) & np.isfinite(y)
                x, y = x[mask], y[mask]
                all_x.extend(x)
                all_y.extend(y)
                key = (app, band)
                mcol = colors[key]
                mmkr = markers[key]
                if overlay:
                    ax.scatter(x, y, color=mcol, marker=mmkr, s=45, alpha=0.80,
                               label=f"{app} {band}")
                else:
                    fig, ax_ = plt.subplots(figsize=(8, 6))
                    ax_.scatter(x, y, color=mcol, marker=mmkr, s=45, alpha=0.80,
                                label=f"{app} {band}")
                    figs.append(fig)
                    axs.append(ax_)
                # Per-line fit
                if len(x) > 1:
                    slope, intercept, *_ = linregress(x, y)
                    slopes_dict[(app, band)] = slope
                    xfit = np.linspace(np.nanmin(x), np.nanmax(x), 80)
                    yfit = slope * xfit + intercept
                    if overlay:
                        ax.plot(xfit, yfit, color=mcol, lw=2, alpha=0.7,
                                label=f'{app} {band} fit (slope={slope:.2f})')
                    else:
                        ax_.plot(xfit, yfit, color=mcol, lw=2, alpha=0.7,
                                 label=f'{app} {band} fit (slope={slope:.2f})')
            # All together fit
            all_x = np.asarray(all_x)
            all_y = np.asarray(all_y)
            mask = np.isfinite(all_x) & np.isfinite(all_y)
            if mask.sum() > 1 and overlay:
                slope, intercept, *_ = linregress(all_x[mask], all_y[mask])
                xfit = np.linspace(np.nanmin(all_x[mask]), np.nanmax(all_x[mask]), 150)
                yfit = slope * xfit + intercept
                ax.plot(xfit, yfit, 'k--', lw=1.5, alpha=0.9,
                        label=f'Band {band} All fit (slope={slope:.2f})')
                slopes_dict[f"{band}_All"] = slope

    elif compare_mode == 'per_approach_across_bands':
        for app in approaches_to_compare:
            if bands_to_compare is None:
                bands_this = list(catalog_nested_dict[app].keys())
            else:
                bands_this = [b for b in bands_to_compare if b in catalog_nested_dict[app]]
            all_x, all_y = [], []
            for band in bands_this:
                df = catalog_nested_dict[app][band]
                eirp_col = get_eirp_col(app)
                x = df[eirp_col].to_numpy()
                y = df['log_TR'].to_numpy()
                mask = np.isfinite(x) & np.isfinite(y)
                x, y = x[mask], y[mask]
                all_x.extend(x)
                all_y.extend(y)
                key = (app, band)
                mcol = colors[key]
                mmkr = markers[key]
                if overlay:
                    ax.scatter(x, y, color=mcol, marker=mmkr, s=45, alpha=0.80,
                               label=f"{app} {band}")
                else:
                    fig, ax_ = plt.subplots(figsize=(8, 6))
                    ax_.scatter(x, y, color=mcol, marker=mmkr, s=45, alpha=0.80,
                                label=f"{app} {band}")
                    figs.append(fig)
                    axs.append(ax_)
                # Per-line fit
                if len(x) > 1:
                    slope, intercept, *_ = linregress(x, y)
                    slopes_dict[(app, band)] = slope
                    xfit = np.linspace(np.nanmin(x), np.nanmax(x), 80)
                    yfit = slope * xfit + intercept
                    if overlay:
                        ax.plot(xfit, yfit, color=mcol, lw=2, alpha=0.7,
                                label=f'{app} {band} fit (slope={slope:.2f})')
                    else:
                        ax_.plot(xfit, yfit, color=mcol, lw=2, alpha=0.7,
                                 label=f'{app} {band} fit (slope={slope:.2f})')
            # All together fit
            all_x = np.asarray(all_x)
            all_y = np.asarray(all_y)
            mask = np.isfinite(all_x) & np.isfinite(all_y)
            if mask.sum() > 1 and overlay:
                slope, intercept, *_ = linregress(all_x[mask], all_y[mask])
                xfit = np.linspace(np.nanmin(all_x[mask]), np.nanmax(all_x[mask]), 150)
                yfit = slope * xfit + intercept
                ax.plot(xfit, yfit, 'k--', lw=1.5, alpha=0.9,
                        label=f'{app} All bands fit (slope={slope:.2f})')
                slopes_dict[f"{app}_All"] = slope

    elif compare_mode == 'combined_across_approaches':
        # Assumes dicts like {'Gaia': {'all': ...}, ...}
        all_x, all_y = [], []
        for app in approaches_to_compare:
            df = catalog_nested_dict[app]['all']
            eirp_col = get_eirp_col(app)
            x = df[eirp_col].to_numpy()
            y = df['log_TR'].to_numpy()
            mask = np.isfinite(x) & np.isfinite(y)
            x, y = x[mask], y[mask]
            all_x.extend(x)
            all_y.extend(y)
            mcol = colors.get((app, 'all'), f'C{approaches_to_compare.index(app)}')
            mmkr = markers.get((app, 'all'), 'o')
            if overlay:
                ax.scatter(x, y, color=mcol, marker=mmkr, s=50, alpha=0.85, label=app)
            else:
                fig, ax_ = plt.subplots(figsize=(8,6))
                ax_.scatter(x, y, color=mcol, marker=mmkr, s=45, alpha=0.80,
                            label=app)
                figs.append(fig)
                axs.append(ax_)
            if len(x) > 1:
                slope, intercept, *_ = linregress(x, y)
                slopes_dict[app] = slope
                xfit = np.linspace(np.nanmin(x), np.nanmax(x), 90)
                yfit = slope * xfit + intercept
                if overlay:
                    ax.plot(xfit, yfit, color=mcol, lw=2, alpha=0.7,
                            label=f'{app} fit (slope={slope:.2f})')
                else:
                    ax_.plot(xfit, yfit, color=mcol, lw=2, alpha=0.7,
                             label=f'{app} fit (slope={slope:.2f})')
        # All together
        all_x = np.asarray(all_x)
        all_y = np.asarray(all_y)
        mask = np.isfinite(all_x) & np.isfinite(all_y)
        if mask.sum() > 1 and overlay:
            slope, intercept, *_ = linregress(all_x[mask], all_y[mask])
            xfit = np.linspace(np.nanmin(all_x[mask]), np.nanmax(all_x[mask]), 180)
            yfit = slope * xfit + intercept
            ax.plot(xfit, yfit, 'k--', lw=1.7, alpha=0.98,
                    label=f'All approaches fit (slope={slope:.2f})')
            slopes_dict["All"] = slope

    # Draw vertical/benchmark lines
    if overlay:
        for xpos, vcolor, label in vlines:
            ax.axvline(x=xpos, ls='--', color=vcolor, lw=1.2, alpha=0.55)
            ax.text(xpos+0.10, ax.get_ylim()[0]+0.01, label, color=vcolor, rotation=90, va='bottom', ha='left', fontsize=10, fontweight='bold', alpha=0.65)
        ax.set_xlabel(r'$\log_{10}\ \mathrm{EIRP_{min}}\ \mathrm{(W)}$')
        ax.set_ylabel(r'$\log_{10}\ T\!R$')
        ax.legend(fontsize=9)
        ax.set_title("log(TR) vs. log(EIRP$_{min}$)")
        ax.grid(True, alpha=0.22)
        plt.tight_layout()
        if show: plt.show()

    if not overlay:
        for ax_ in axs:
            ax_.set_xlabel(r'$\log_{10}\ \mathrm{EIRP_{min}}\ \mathrm{(W)}$')
            ax_.set_ylabel(r'$\log_{10}\ T\!R$')
            ax_.legend(fontsize=9)
            ax_.set_title("log(TR) vs. log(EIRP$_{min}$)")
            ax_.grid(True, alpha=0.22)
            plt.tight_layout()
            if show: ax_.figure.show()

    if return_slopes:
        if overlay:
            return (fig, ax, slopes_dict)
        else:
            return (figs, axs, slopes_dict)
    else:
        if overlay:
            return (fig, ax)
        else:
            return (figs, axs)


# CMD taking into account overlay and multiple bands

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_density_colored_cmd_catalog_compare(
    catalog_dict,
    bands_to_compare=None,
    overlay=True,
    cmap_names=None,
    point_size=8,
    xlim=None, ylim=None,
    show=True,
    title=None
):
    """
    Plot single or overlaid density-colored CMDs for Gaia & SynthPop, per or across bands.
    If overlay=True, overlays all selected CMDs on one plot, with one matching colorbar per scatter.
    Otherwise, creates a separate plot for each combination.

    Parameters
    ----------
    catalog_dict : dict
        Dict: {'Gaia': {'L': df_gaia_L, ...}, 'SynthPop': {'L': df_syn_L, ...}, ...}
        Each inner dict contains DataFrames per band, e.g., {"L": <df>, "S": <df>}
    bands_to_compare : list or None
        Which bands to show (if None, all bands present in any catalog are shown).
    overlay : bool
        If True, overlay all provided CMDs on one plot, each with its own colorbar.
        If False, separate figure for each.
    cmap_names : dict or None
        Mapping: (catalog, band) -> matplotlib colormap name.
        Example: {('Gaia','L'): 'viridis', ('SynthPop','L'): 'plasma', ...}
        If None, uses standard choices.
    point_size : int, default 8
        Size of points.
    xlim, ylim : optional
        Plotting limits as in matplotlib.
    show : bool
        If True, display figure(s) immediately.
    title : str or None
        Plot title (used for overlay; per-panel title otherwise).
        If none -> default is 'Gaia/SynthPop L/S CMD (Density Colored)'

    Returns
    -------
    - If overlay: (fig, ax)
    - Else: lists of (fig, ax)
    """

    if cmap_names is None:
        cmap_names = {('Gaia', 'L'): 'viridis', ('SynthPop', 'L'): 'plasma',
                      ('Gaia', 'S'): 'cividis', ('SynthPop', 'S'): 'inferno'}

    approaches = list(catalog_dict.keys())
    bands_available = set()
    for app in approaches:
        bands_available |= set(catalog_dict[app].keys())
    if bands_to_compare is None:
        bands_to_show = list(sorted(bands_available))
    else:
        bands_to_show = [b for b in bands_to_compare if b in bands_available]

    if overlay:
        fig, ax = plt.subplots(figsize=(7, 9))
        colorbars = []
        label_list = []
    else:
        figs, axs = [], []

    for app in approaches:
        for band in bands_to_show:
            if band not in catalog_dict[app]:
                continue
            df = catalog_dict[app][band]
            # Gaia/SynthPop logic
            if app.lower() == "gaia":
                if not all(col in df.columns for col in ['bp_rp','abs_g_photogeo']):
                    continue
                x = df['bp_rp']
                y = df['abs_g_photogeo']
                xlab, ylab = 'BP − RP (mag)', 'Abs G-band Mag (photogeo)'
            else:
                if not all(c in df.columns for c in ['Gaia_BP_EDR3', 'Gaia_RP_EDR3', 'Gaia_G_EDR3', 'Dist_pc']):
                    continue
                x = df['Gaia_BP_EDR3'] - df['Gaia_RP_EDR3']
                y = df['Gaia_G_EDR3'] - 5 * np.log10(df['Dist_pc']) + 5
                xlab, ylab = "BP − RP (mag)", "Abs G-band Mag (SynthPop)"
            mask = x.notna() & y.notna() & np.isfinite(x) & np.isfinite(y)
            x_clean, y_clean = x[mask], y[mask]
            if len(x_clean) < 3:
                continue

            # Compute density for coloring
            xy = np.vstack([x_clean, y_clean])
            try:
                density = gaussian_kde(xy)(xy)
                show_density = True
            except Exception:
                density = np.ones_like(x_clean)
                show_density = False

            sqrt_density = np.sqrt(density)
            cmap = plt.colormaps.get_cmap(cmap_names.get((app, band), 'plasma'))
            norm = mcolors.Normalize(vmin=sqrt_density.min(), vmax=sqrt_density.max())
            label = f"{app} {band}"

            if overlay:
                sc = ax.scatter(
                    x_clean, y_clean, c=sqrt_density, s=point_size, cmap=cmap,
                    norm=norm, edgecolor='none', alpha=0.92, label=label
                )
                colorbars.append({"handle": sc, "label": label, "cmap": cmap})
                label_list.append(label)
            else:
                fig, ax_ = plt.subplots(figsize=(7, 9))
                sc = ax_.scatter(x_clean, y_clean, c=sqrt_density, s=point_size, cmap=cmap,
                                 norm=norm, edgecolor='none', alpha=0.92, label=label)
                ax_.invert_yaxis()
                ax_.set_xlabel(xlab)
                ax_.set_ylabel(ylab)
                ax_.set_title(title or f"{app} {band} CMD (Density Colored)")
                if show_density:
                    fig.colorbar(sc, ax=ax_, label='√(Relative Density)')
                if xlim: ax_.set_xlim(xlim)
                if ylim: ax_.set_ylim(ylim)
                ax_.grid(True, alpha=0.15)
                plt.tight_layout()
                figs.append(fig)
                axs.append(ax_)

    if overlay:
        ax.invert_yaxis()
        ax.set_xlabel(xlab)
        ax.set_ylabel(ylab)
        ax.set_title(title or f"CMD Comparison: " + ", ".join(label_list))
        ax.legend()
        if xlim: ax.set_xlim(xlim)
        if ylim: ax.set_ylim(ylim)
        ax.grid(True, alpha=0.15)
        plt.tight_layout()

        # -------- Add multiple colorbars for overlays --------
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        divider = make_axes_locatable(ax)
        # Place each colorbar to right, incrementally offset
        n_cbars = len(colorbars)
        pad = 0.5
        for i, cbar_info in enumerate(colorbars):
            # Each bar sits further to the right by (pad + i * perbar_pad)
            cax = divider.append_axes("right", size="3.7%", pad=pad + i*0.1)
            cb = fig.colorbar(cbar_info["handle"], cax=cax)
            cb.set_label(cbar_info["label"], fontsize=9)
            cb.ax.tick_params(labelsize=8)

        if show:
            plt.show()
        return fig, ax
    else:
        if show:
            for fig in figs:
                fig.show()
        return figs, axs
